In [1]:
%config InlineBackend.rc = {"figure.dpi": 72, "figure.figsize": (6.0, 4.0)}
%matplotlib ipympl

import abtem
import ase
import matplotlib.pyplot as plt
import numpy as np
from ase.cluster import Decahedron

In [33]:
substrate = ase.build.bulk("C", cubic=True)

substrate *= (8, 8, 12)

bl = 1.54  # Bond length
rng = np.random.default_rng(seed=10)
substrate.positions[:] += rng.normal(size=(len(substrate), 3)) * 0.5 * bl

substrate.wrap()

In [34]:
p = 4
q = 2
r = 1

nanoparticle = Decahedron("Au", p, q, r, latticeconstant=None)
nanoparticle.rotate(30, "x", center="cop")

nanoparticle.center(vacuum=1)

nanoparticle.set_cell(substrate.cell)
nanoparticle.center(axis=(0, 1))
nanoparticle.translate((0, 0, substrate.cell[2, 2] - 2))

atoms = nanoparticle + substrate

atoms.center(axis=2, vacuum=2)

In [35]:
wave = abtem.PlaneWave(energy=100e3)

potential = abtem.Potential(atoms, sampling=0.1, exit_planes=1, slice_thickness=.5)

exit_wave = wave.multislice(potential).compute()#.ensure_lazy()


[########################################] | 100% Completed | 2.40 ss


In [40]:
fig, ax = plt.subplots()

visualization = exit_wave.show(display=False, vmin=0, vmax=6, cmap="viridis", ax=ax, cbar=True)

visualization.axis_off(spines=False)

visualization.adjust_tight_bbox()

animation = visualization.animate(adjust_scale=False, interval=200)

In [41]:
animation.save("animation.gif", writer="pillow")